In [17]:
import pandas as pd
import requests
import time
import json
import re

In [ ]:
planning_area,year,no_religion,buddhism,taoism,islam,hinduism,sikhism,catholic_christian,other_christians,other_religions	
planning_area,year,chinese, malays, indian	
planning_area,year,english,mandarin,chinese_dialects,malay,tamil	


Reading

In [2]:
import plotly.express as px
df_stat = pd.read_csv('hdb_stats_final.csv')
df_coords = pd.read_csv('hdb_coords.csv')

/Users/rachelkindangen/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1,2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [169]:
df_ethnic = pd.read_csv('ethnicity_percent.csv')
df_price = pd.read_csv('cleaning_output', names = ['planning_area','floor_area_sqm','year','resale_price'])

/Users/rachelkindangen/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1,2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [124]:
df_stat = df_stat.iloc[:,1:]

In [187]:
df_religion = df_stat.iloc[:,26:35]

In [190]:
df_religion['total'] = df_religion.sum(axis=1)

In [192]:
religions = ["no_religion","buddhism",'taoism',"islam","hinduism","sikhism","catholic_christian","other_christians","other_religions"]
for religion in religions:
    new_col = religion + " percent"
    df_religion[new_col] = df_religion[religion]/df_religion['total']


In [194]:
df_religion['planning_area'] = df_stat['planning_area']
df_religion['year'] = df_stat['year']


In [197]:
df_religion = df_religion.iloc[:,11:21]

In [200]:
df_religion['year'] = df_religion['year'].astype('string')

In [201]:
df_religion.head()

,buddhism percent,taoism percent,islam percent,hinduism percent,sikhism percent,catholic_christian percent,other_christians percent,other_religions percent,planning_area,year
0,0.394584,0.071294,0.203597,0.023754,0.003108,0.053340,0.103118,0.002773,ANG MO KIO,2000
1,0.394870,0.068578,0.064612,0.046662,0.004570,0.088985,0.147287,0.002759,BEDOK,2000
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BISHAN,2000
3,0.413777,0.072108,0.175542,0.051830,0.003755,0.036584,0.095873,0.002963,BOON LAY,2000
4,0.473171,0.110231,0.075798,0.048576,0.004522,0.032143,0.088983,0.001445,BUKIT BATOK,2000


In [170]:
df_ethnic = df_ethnic.iloc[:, 1:]

In [171]:
df_ethnic

,chinese_percent,malays_percent,indian_percent,planning_area,year
0,0.743809,0.184355,0.071837,ANG MO KIO,2000
1,0.860790,0.057782,0.081428,BEDOK,2000
2,NaN,NaN,NaN,BISHAN,2000
3,NaN,NaN,NaN,BUKIT TIMAH,2000
4,NaN,NaN,NaN,CENTRAL WATER CATCHMENT,2000
...,...,...,...,...,...
1024281,0.874034,0.037124,0.088841,NORTH-EASTERN ISLANDS,2020
1024282,0.699108,0.207956,0.092936,NOVENA,2020
1024283,0.815545,0.010441,0.174014,PAYA LEBAR,2020
1024284,0.762805,0.048857,0.188337,RIVER VALLEY,2020


In [172]:
df_price= df_price[:-1]

In [173]:
df_price['resale_price'] = df_price['resale_price'].apply(lambda x: re.sub(r'\s+', '', x) if type(x)==str else x )




In [174]:
df_price['avg_price_per_sqm'] = df_price['resale_price'].astype(float) / df_price['floor_area_sqm'].astype(float)

In [175]:
df_price_avg = df_price.groupby(by = ['planning_area', 'year'])['avg_price_per_sqm'].mean().reset_index()

In [176]:
df_price_avg

,planning_area,year,avg_price_per_sqm
0,ANG MO KIO,1976,2419.740758
1,ANG MO KIO,1977,2531.210881
2,ANG MO KIO,1978,2530.513774
3,ANG MO KIO,1979,2478.332208
4,ANG MO KIO,1980,2483.200036
...,...,...,...
894,YISHUN,1993,3084.165519
895,YISHUN,1995,3114.822921
896,YISHUN,1999,5355.729167
897,YISHUN,2000,3883.287354


In [177]:
years_to_keep = ['1999','2000','2001', '2008', '2010','2009', '2014','2015','2016', '2018','2021','2020']
df_price_avg = df_price_avg[df_price_avg['year'].isin(years_to_keep)]

In [178]:
df_ethnic.head()

,chinese_percent,malays_percent,indian_percent,planning_area,year
0,0.743809,0.184355,0.071837,ANG MO KIO,2000
1,0.860790,0.057782,0.081428,BEDOK,2000
2,NaN,NaN,NaN,BISHAN,2000
3,NaN,NaN,NaN,BUKIT TIMAH,2000
4,NaN,NaN,NaN,CENTRAL WATER CATCHMENT,2000


In [179]:
df_join = df_price_avg.merge(df_ethnicity, how = "inner", left_on=['planning_area','year'], right_on = ['planning_area','year'])



In [182]:
df_join[~df_join['chinese_percent'].isnull()].sort_values(by = 'avg_price_per_sqm')[df_join['year'].isin(['2000'])]

/var/folders/08/1gbml__92sdfdblw8tw5pyd80000gn/T/ipykernel_3361/3509427280.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_join[~df_join['chinese_percent'].isnull()].sort_values(by = 'avg_price_per_sqm')[df_join['year'].isin(['2000'])]


,planning_area,year,avg_price_per_sqm,chinese_percent,malays_percent,indian_percent
6075,CHOA CHU KANG,2000,3242.740134,0.767684,0.172828,0.059488
17216,WOODLANDS,2000,3327.626688,0.717992,0.170307,0.111701
11141,SEMBAWANG,2000,3501.275892,0.805242,0.137828,0.056930
12154,SENGKANG,2000,3754.982190,0.859033,0.056499,0.084467
1012,BEDOK,2000,3973.001019,0.860790,0.057782,0.081428
7090,JURONG WEST,2000,3973.238149,0.948329,0.006965,0.044706
7089,JURONG EAST,2000,4170.698032,0.791343,0.126739,0.081918
2026,BUKIT MERAH,2000,4962.841447,0.744842,0.187772,0.067387
6076,GEYLANG,2000,5461.909873,0.841789,0.097060,0.061151
10128,QUEENSTOWN,2000,5802.378936,0.723384,0.195933,0.080683


In [202]:
df_join_1 = df_price_avg.merge(df_religion, how = "inner", left_on=['planning_area','year'], right_on = ['planning_area','year'])


In [207]:
df_join_1[~df_join_1['buddhism percent'].isnull()].sort_values(by = 'avg_price_per_sqm')[df_join_1['year'].isin(['2000'])]

/var/folders/08/1gbml__92sdfdblw8tw5pyd80000gn/T/ipykernel_3361/223101559.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_join_1[~df_join_1['buddhism percent'].isnull()].sort_values(by = 'avg_price_per_sqm')[df_join_1['year'].isin(['2000'])]


,planning_area,year,avg_price_per_sqm,buddhism percent,taoism percent,islam percent,hinduism percent,sikhism percent,catholic_christian percent,other_christians percent,other_religions percent
9,CHOA CHU KANG,2000,3242.740134,0.461590,0.091327,0.168092,0.031984,0.003175,0.036528,0.077176,0.002162
29,WOODLANDS,2000,3327.626688,0.435894,0.092617,0.173593,0.066661,0.003431,0.035278,0.074224,0.002629
20,SEMBAWANG,2000,3501.275892,0.504195,0.081891,0.134652,0.033218,0.002947,0.039272,0.068348,0.002841
22,SENGKANG,2000,3754.982190,0.424608,0.075519,0.060383,0.047416,0.003635,0.087073,0.130248,0.002317
31,YISHUN,2000,3883.287354,0.569607,0.095238,0.045017,0.027322,0.000000,0.041374,0.050481,0.000000
1,BEDOK,2000,3973.001019,0.394870,0.068578,0.064612,0.046662,0.004570,0.088985,0.147287,0.002759
13,JURONG WEST,2000,3973.238149,0.270976,0.031774,0.020723,0.022554,0.003016,0.106870,0.283385,0.005622
12,JURONG EAST,2000,4170.698032,0.413126,0.072089,0.125719,0.035235,0.003970,0.054123,0.121142,0.002250
4,BUKIT MERAH,2000,4962.841447,0.445612,0.075946,0.186372,0.041099,0.002518,0.028017,0.079438,0.002499
10,GEYLANG,2000,5461.909873,0.487633,0.094525,0.098339,0.034614,0.003188,0.065072,0.080440,0.004421


Graph

In [130]:
# pln_area_polygons = pd.DataFrame(result)

In [146]:
# pln_area_polygons['geojson'] = pln_area_polygons['geojson'].apply(lambda x: json.loads(x)['coordinates'] if x is not None else 0)



In [158]:
# fig = px.scatter_geo(y, geojson = "geojson", size = "english",
#                      color = "planning_area", projection="natural earth")
# fig.show()

In [ ]:
#join on ethnicity data to check diversity of religions
#find increases in population percentages over time compare to changes in prices over time
#clustering = races reaching the population quota in a neighborhood 
#how do prices impact racial clustering?
#how does racial clustering impact prices? 